In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import sys
import time
from datetime import datetime, date, time

In [2]:
l_parse_date_cols = ['dt_prediction_date', 'dt_target_date', 'dt_flight_date']
l_target_cols = ['num_pax_000_014_mins_before_sdt', 'num_pax_015_029_mins_before_sdt', 'num_pax_030_044_mins_before_sdt', 'num_pax_045_059_mins_before_sdt', 'num_pax_060_074_mins_before_sdt', 'num_pax_075_089_mins_before_sdt', 'num_pax_090_104_mins_before_sdt', 'num_pax_105_119_mins_before_sdt',  'num_pax_120_134_mins_before_sdt', 'num_pax_135_149_mins_before_sdt', 'num_pax_150_164_mins_before_sdt', 'num_pax_165_179_mins_before_sdt',  'num_pax_180_194_mins_before_sdt', 'num_pax_195_209_mins_before_sdt', 'num_pax_210_224_mins_before_sdt', 'num_pax_225_239_mins_before_sdt', 'num_pax_240plus_mins_before_sdt']


In [3]:
train = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\data_20161116\train.csv", parse_dates = l_parse_date_cols)
train = train.set_index('id')
test = pd.read_csv(r"C:\Users\piush\Desktop\Dataset\data_20161116\test.csv", parse_dates = l_parse_date_cols)
test = test.set_index('id')

In [4]:
test2 = test[pd.isnull(test).any(axis=1)]

In [5]:
target = train[l_target_cols]

In [6]:
df1 = train.drop(l_target_cols,axis = 1)

In [7]:
# Concatenate the 2 data sets
df = df1.append(test2, ignore_index = True)

In [8]:
def is_outlier(points, thresh = 3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [9]:
cats = []
for col in df.columns.values:
    if df[col].dtype == 'object':
        cats.append(col)

In [10]:
df_cont = df.drop(cats, axis=1)
df_cat = df[cats]

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from scipy.stats import skew


In [12]:
cats = []
for col in df.columns.values:
    if df[col].dtype == 'object':
        cats.append(col)

In [13]:
df_cont = df.drop(cats, axis=1)
df_cat = df[cats]

In [14]:
for col in df_cont.columns.values:
    if np.sum(df_cont[col].isnull()) > 50:
        #print("Removing Column: {}".format(col))
        df_cont = df_cont.drop(col, axis = 1)
    elif np.sum(df_cont[col].isnull()) > 0:
        #print("Replacing with Median: {}".format(col))
        median = df_cont[col].median()
        idx = np.where(df_cont[col].isnull())[0]
        df_cont[col].iloc[idx] = median
        
        
        outliers = np.where(is_outlier(df_cont[col]))
        df_cont[col].iloc[outliers] = median
        
               
        if skew(df_cont[col]) > 0.75:
            #print("Skewness Detected: {}".format(col))
            df_cont[col] = np.log(df_cont[col])
            df_cont[col] = df_cont[col].apply(lambda x: 0 if x == -np.inf else x)
        
        df_cont[col] = Normalizer().fit_transform(df_cont[col].reshape(1,-1))[0]

In [15]:
for col in df_cat.columns.values:
    if np.sum(df_cat[col].isnull()) > 50:
        df_cat = df_cat.drop(col, axis = 1)
        continue
    elif np.sum(df_cat[col].isnull()) > 0:
        df_cat[col] = df_cat[col].fillna('MIA')
        
    df_cat[col] = LabelEncoder().fit_transform(df_cat[col])
    
    num_cols = df_cat[col].max()
    for i in range(num_cols):
        col_name = col + '_' + str(i)
        df_cat[col_name] = df_cat[col].apply(lambda x: 1 if x == i else 0)
        
    df_cat = df_cat.drop(col, axis = 1)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
df_new = df_cont.join(df_cat)

In [17]:
X = df_new[:train.shape[0]]
test1 = df_new[train.shape[0]:]


In [18]:
X = X.drop(['dt_flight_date','dt_prediction_date','dt_target_date'],axis = 1)


In [19]:
test1 = test1.drop(['dt_flight_date','dt_prediction_date','dt_target_date'],axis = 1)

In [20]:
print (str(len(X))+" rows for training set")
print (str(len(test1))+" rows for test set")
print (str(len(target))+" rows for target")

98043 rows for training set
4577 rows for test set
98043 rows for target


In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf_random = RandomForestRegressor(n_estimators=500, n_jobs=-1)

clf_random.fit(X, target)
y_pred = clf_random.predict(test1)
